<h1>Time Series</h1>

<h2>The objective of this notebook is: </h2>
<ol>
<li>In the first part: understand Time series forecasting with ARIMA model in python, and fine tune the hyper-parameters to get the least possible error.</li>
<li> In the second part: Apply CNN from scratch to the same dataset.</li>
<li> Apply Facebook prophet to the same dataset.</li>
</ol>

<h2>The dataset</h2>

Build a model to forecast the demand (passenger traffic) in Airplanes.<br> 
The data is classified in date/time and the passengers travelling per month

<h2>What is Time series analysis?</h2>

A. Time Series is a series of observations taken at specified time intervals usually equal intervals. <br>
 

<h2>We don't need to apply Time series in atleast the following 2 cases:</h2>
<ul>
<li> 
a) The dependant variable(y) (that is supposed to vary with time) is constant.<br>
    Eq: y=f(x)=4, a line parallel to x-axis(time) will always remain the same.</li> 
<li> b) The dependant variable(y) represent values that can be denoted as a mathematical function.<br> Eq: sin(x), log(x), Polynomials etc. Thus, we can directly get value at some time using the function itself. No need of forecasting.</li> 
    </ul>

<h2>There are 4 componentsof Time Series:</h2>
<ul>
<li>
a) Trend - Upward & downward movement of the data with time over a large period of time. Eq: Appreciation of Dollar vs rupee.</li>
<li>b) Seasonality - seasonal variances. Eq: Ice cream sales increases in Summer only</li>
<li>c) Noise or Irregularity - Spikes & troughs at random intervals</li>
<li>d) Cyclicity - behavior that repeats itself after large interval of time, like months, years etc.</li>
</ul>

<h2>Why does Time Series(TS) need to be stationary?</h2>
<h3>It is because of the following reasons:</h3>
<ul>
<li>a) If a TS has a particular behavior over a time interval, then there's a high probability that over a different interval, it will have same behavior, provided TS is stationary. This helps in forecasting accurately.</li>
<li>b) Theories & Mathematical formulas ae more mature & easier to apply for as TS which is stationary.</li>
</ul>

<h2>Tests to check if a series is stationary or not</h2>
<h3> There are 2 ways to check for Stationarity of a TS:</h3>
<ul>
 <li>a) Rolling Statistics - Plot the moving avg or moving standard deviation to see if it varies with time. Its a visual technique.</li>
<li>b) ADCF Test - Augmented Dickey–Fuller test is used to gives us various values that can help in identifying stationarity. The Null hypothesis says that a TS is non-stationary. It comprises of a Test Statistics & some critical values for some confidence levels. If the Test statistics is less than the critical values, we can reject the null hypothesis & say that the series is stationary. THE ADCF test also gives us a p-value. Acc to the null hypothesis, lower values of p is better.</li>
     </ul>

<h1> Time Searis usinf LSTM</h1>

## 1- Load the data

### Importing all the required libraries for analysis

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import itertools
import numpy as np
import random
from statsmodels.tsa.stattools import adfuller

import statsmodels.api as sm
# prophet by Facebook
# from fbprophet import Prophet
# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

### Reading the dataset 
 Creating the 'Date' as Index for data and viewing the dataset

In [6]:
path = "../Data/AirPassengers.csv" 
df = pd.read_csv(path,parse_dates=[0])

## 2- Preprocessing

### convert from string to datetime

In [7]:
df['Month'] = pd.to_datetime(df['Month'],infer_datetime_format=True)
df = df.set_index(['Month'])
df.head()

#Passengers
Month                  
1949-01-01          112
1949-02-01          118
1949-03-01          132
1949-04-01          129
1949-05-01          121

### Visualizing the Time Series plot for the number of Air Passengers

In [8]:
## Plot the Time series data
fig, ax = plt.subplots(figsize=(20,7))
a = sns.lineplot(x="Month", y="#Passengers", data=df)
a.set_title("Number of air passengers",fontsize=15)
plt.show()

<h2>Scaling the Data</h2>

This means that you're transforming your data so that it fits within a specific scale, like 0-100 or 0-1. <br>
You want to scale data when you're using methods based on measures of how far apart data points, like support vector machines, or SVM or k-nearest neighbors, or KNN.

<h2>Do I have to scale/normalize my training data for LSTM Classification, even if I only have one feature?</h2>

It's still advisable to do so because your randomly initialised weights may not be able to get along with feature scale very well, and may cause the activations to saturate.

In [9]:
#data_Scaling
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(df)

In [10]:
df

array([[0.01544402],
       [0.02702703],
       [0.05405405],
       [0.04826255],
       [0.03281853],
       [0.05984556],
       [0.08494208],
       [0.08494208],
       [0.06177606],
       [0.02895753],
       [0.        ],
       [0.02702703],
       [0.02123552],
       [0.04247104],
       [0.07142857],
       [0.05984556],
       [0.04054054],
       [0.08687259],
       [0.12741313],
       [0.12741313],
       [0.1042471 ],
       [0.05598456],
       [0.01930502],
       [0.06949807],
       [0.07915058],
       [0.08880309],
       [0.14285714],
       [0.11389961],
       [0.13127413],
       [0.14285714],
       [0.18339768],
       [0.18339768],
       [0.15444015],
       [0.11196911],
       [0.08108108],
       [0.11969112],
       [0.12934363],
       [0.14671815],
       [0.17181467],
       [0.14864865],
       [0.15250965],
       [0.22007722],
       [0.24324324],
       [0.26640927],
       [0.2027027 ],
       [0.16795367],
       [0.13127413],
       [0.173

## Creating train and test partition


In [11]:
#Creating train and test partition
train = int(len(df)*0.75)
test = len(df)-train



train_data,test_data=df[0:train,:],df[train:len(df),:1]

## Converting an array of values into a dataset matrix


In [12]:
# converting an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [13]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 4
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

y_train

array([0.03281853, 0.05984556, 0.08494208, 0.08494208, 0.06177606,
       0.02895753, 0.        , 0.02702703, 0.02123552, 0.04247104,
       0.07142857, 0.05984556, 0.04054054, 0.08687259, 0.12741313,
       0.12741313, 0.1042471 , 0.05598456, 0.01930502, 0.06949807,
       0.07915058, 0.08880309, 0.14285714, 0.11389961, 0.13127413,
       0.14285714, 0.18339768, 0.18339768, 0.15444015, 0.11196911,
       0.08108108, 0.11969112, 0.12934363, 0.14671815, 0.17181467,
       0.14864865, 0.15250965, 0.22007722, 0.24324324, 0.26640927,
       0.2027027 , 0.16795367, 0.13127413, 0.17374517, 0.17760618,
       0.17760618, 0.25482625, 0.25289575, 0.24131274, 0.26833977,
       0.30888031, 0.32432432, 0.25675676, 0.20656371, 0.14671815,
       0.18725869, 0.19305019, 0.16216216, 0.25289575, 0.23745174,
       0.25096525, 0.30888031, 0.38223938, 0.36486486, 0.2992278 ,
       0.24131274, 0.19111969, 0.24131274, 0.26640927, 0.24903475,
       0.31467181, 0.31853282, 0.32046332, 0.40733591, 0.50193

In [14]:
X_train.shape, y_train.shape


((103, 4), (103,))

In [15]:
# reshape input to be [sample, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0], 4, 1)
X_test = X_test.reshape(X_test.shape[0], 4, 1)

X_train.shape

(103, 4, 1)

In [16]:
# !pip install numpy==1.19.5


In [17]:
### Create the LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(4,1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=1,verbose=1)

Epoch 1/100
103/103 [==============================] - 10s 33ms/step - loss: 0.0190 - val_loss: 0.0330
Epoch 2/100
103/103 [==============================] - 1s 7ms/step - loss: 0.0093 - val_loss: 0.0624
Epoch 3/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0085 - val_loss: 0.0333
Epoch 4/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0092 - val_loss: 0.0314
Epoch 5/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0090 - val_loss: 0.0267
Epoch 6/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0067 - val_loss: 0.0358
Epoch 7/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0071 - val_loss: 0.0272
Epoch 8/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0067 - val_loss: 0.0318
Epoch 9/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0067 - val_loss: 0.0261
Epoch 10/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0064 - val_

103/103 [==============================] - 1s 6ms/step - loss: 0.0021 - val_loss: 0.0213
Epoch 82/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0021 - val_loss: 0.0122
Epoch 83/100
103/103 [==============================] - 1s 7ms/step - loss: 0.0020 - val_loss: 0.0092
Epoch 84/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0024 - val_loss: 0.0183
Epoch 85/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0023 - val_loss: 0.0075
Epoch 86/100
103/103 [==============================] - 1s 7ms/step - loss: 0.0024 - val_loss: 0.0100
Epoch 87/100
103/103 [==============================] - 1s 7ms/step - loss: 0.0022 - val_loss: 0.0116
Epoch 88/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0021 - val_loss: 0.0079
Epoch 89/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0023 - val_loss: 0.0122
Epoch 90/100
103/103 [==============================] - 1s 6ms/step - loss: 0.0021 - val_loss: 

In [18]:
#Model Prediction
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

test_predict

array([[0.48751956],
       [0.57013786],
       [0.7711164 ],
       [0.8222227 ],
       [0.7646117 ],
       [0.41802177],
       [0.51955247],
       [0.41187224],
       [0.5413444 ],
       [0.56111574],
       [0.4789348 ],
       [0.71269524],
       [0.56194675],
       [0.68756664],
       [0.8027717 ],
       [0.95088315],
       [0.7995095 ],
       [0.5015667 ],
       [0.57755506],
       [0.50433105],
       [0.69726706],
       [0.63857067],
       [0.54841936],
       [0.69472086],
       [0.76546156],
       [0.7174901 ],
       [0.93091404],
       [1.0546948 ],
       [0.76458395],
       [0.5798037 ],
       [0.67287433]], dtype=float32)

In [19]:
#Transforming data back to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

test_predict

array([[356.53516],
       [399.33142],
       [503.4383 ],
       [529.9113 ],
       [500.06888],
       [320.53528],
       [373.1282 ],
       [317.34982],
       [384.4164 ],
       [394.65796],
       [352.08823],
       [473.17615],
       [395.08844],
       [460.15955],
       [519.8357 ],
       [596.55743],
       [518.14594],
       [363.81155],
       [403.17352],
       [365.2435 ],
       [465.18436],
       [434.77963],
       [388.08124],
       [463.86542],
       [500.5091 ],
       [475.65988],
       [586.21344],
       [650.33185],
       [500.0545 ],
       [404.33832],
       [452.54892]], dtype=float32)

In [20]:
## Calculate RMSE performance metrics
from sklearn.metrics import mean_squared_error

rmse= np.sqrt(mean_squared_error(y_train,train_predict))

In [21]:
rmse

263.86472485183754

In [22]:
452.56696011597353
### Plotting 
# shift train predictions for plotting
look_back=4
trainPredictPlot = np.empty_like(df)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shift test predictions for plotting
testPredictPlot = np.empty_like(df)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df)-1, :] = test_predict
# plot baseline and predictions
plt.xlabel("Month")
plt.ylabel("Passengers")
plt.title("Passengers Travelled")
plt.plot(scaler.inverse_transform(df)) #original data
plt.plot(testPredictPlot) #test prediction
plt.show()

<h2>Grid search</h2>

For simplicity we are keeping less parameters.

In [23]:
pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc0#egg=keras-tuner-1.0.2rc0

  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc0) to c:\users\gehad\appdata\local\temp\pip-install-vdk9fhdq\keras-tuner-1-0-2rc0
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc0-py3-none-any.whl size=89715 sha256=15edfada8cc55e7326cdda29295e53e7ab3686112797b3854cea71604e4784ee
  Stored in directory: C:\Users\Gehad\AppData\Local\Temp\pip-ephem-wheel-cache-jtf3fq1s\wheels\c5\6f\c4\5bac9bffa7d6deedbf8fc7057d5232ff0ad2c8cef7f560f219
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc0-py3-none-any.whl size=89715 sha256=5325e90f4d01902f3df6d7bcdf0f873cfaca322bb87f7818d0570feb72bb1126
  Stored in directory: c:\users\gehad\appdata\local\pip\cache\wheels\87\f1\b3\b3533b604c98bd41b1efa8739a4bf71f1e64df55050f9daa9a
Successfully built keras-tuner keras-tuner
Note: you may need to restart the kernel to use updated packages.


In [28]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import LSTM,Dense,Dropout


In [30]:
def build_classifier(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(units = 64,input_shape = (33,1)))
    grid_model.add(Dropout(0.4))
    grid_model.add(Dense(1))

    grid_model.compile(loss = 'mse',optimizer = optimizer, metrics = ['mean_squared_error'])
    return grid_model

grid_model = KerasClassifier(build_fn=build_classifier)
parameters = {'batch_size' : [4000,4080],
              'epochs' : [8,10],
              'optimizer' : ['adam','Adadelta'] }

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            scoring = 'accuracy',
                            cv = 2)

In [31]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/8
1/1 [==============================] - 5s 5s/step - loss: 655.2830 - mean_squared_error: 655.2830
Epoch 2/8
1/1 [==============================] - 0s 20ms/step - loss: 654.3879 - mean_squared_error: 654.3879
Epoch 3/8
1/1 [==============================] - 0s 14ms/step - loss: 653.2198 - mean_squared_error: 653.2198
Epoch 4/8
1/1 [==============================] - 0s 12ms/step - loss: 652.4240 - mean_squared_error: 652.4240
Epoch 5/8
1/1 [==============================] - 0s 13ms/step - loss: 652.0303 - mean_squared_error: 652.0303
Epoch 6/8
1/1 [==============================] - 0s 11ms/step - loss: 651.4428 - mean_squared_error: 651.4428
Epoch 7/8
1/1 [==============================] - 0s 8ms/step - loss: 650.2518 - mean_squared_error: 650.2518
Epoch 8/8
1/1 [==============================] - 0s 13ms/step - loss: 649.7808 - mean_squared_error: 649.7808


ValueError: continuous is not supported

In [36]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [35]:
y_pred = model.predict(X_test)
